<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/03_%EB%8B%B5%EB%B3%80%EC%83%9D%EC%84%B1%EC%84%9C%EB%B9%84%EC%8A%A4_%EB%AA%A8%EB%8D%B8%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 데이터파일 불러오기

In [ ]:
df = pd.read_csv(f"{DATA_PATH}train_0920.csv")
df

,고객리뷰,사장댓글,sentiment,food,store,branch
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,김치찌개,,NaN
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,,,NaN
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,,,NaN
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,잡채,,NaN
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,,,NaN
...,...,...,...,...,...,...
18641,맛있어요 배달도 빠르고 국물도 안새고 안전배달 잘먹었습니다,감사드립니다 리뷰님. 찬바람 건강 유의하세요,1,,,
18642,주말되면 생각나서 찾게되는 너무맛있어요 잘먹었습니다~,저희 인생 아귀찜은 신선한 재료와 넉넉한 양으로 고객님께 맛있는 식사를 대접해 드리...,1,아귀찜,인생 아귀찜,
18643,맛있게 잘먹었네요. 배부르네요,맛있게 드셔주셔서 감사합니다,1,,,
18644,맛있게 잘 먹었습니다,맛있게 잘 드셨다니 감사합니다~ 다음에 드실 때도 변함없이 맛있게 드실 수 있도록 ...,1,닭,,


In [ ]:
df['branch'] = df['branch'].fillna("")

# 모델 불러오기

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 75.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.2 MB/s eta 0:00:00


In [ ]:
model_name = "skt/kogpt2-base-v2"

In [ ]:
# AutoModelForCausalLM 인과 관련 사용
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          bos_token='</s>',
                                          eos_token='</s>',
                                          unk_token='<unk>',
                                          pad_token='<pad>',
                                          mask_token='<mask>',
                                          max_len=1024)

# 데이터셋 만들기

In [ ]:
df.head()

,고객리뷰,사장댓글,sentiment,food,store,branch
0,김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고.,안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 ...,0,김치찌개,,
1,양이 너무 많아서 배터질뻔했어요. 서비스도 별로였어요.,배터질뻔하셨다니 정말 안타깝습니다. 양에 대한 조정을 더 신경써야겠습니다. 불편을 ...,0,,,
2,음식이 너무 싱겁고 부족해서 맛이 없었어요. 배달도 늦게 왔고 실망스러웠어요.,음식 맛과 배달 시간에 대한 실망을 드려 죄송합니다. 앞으로 더 신속하고 맛있는 음...,0,,,
3,잡채는 짜다고 하지도 않았는데 맛이 진짜 짜요. 먹기 어려웠어요.,잡채가 짜다는 피드백을 반영하지 못해 죄송합니다. 더 맛있고 조리하기 쉽도록 개선하...,0,잡채,,
4,가격에 비해 음식의 양도 적고 맛도 별로였어요. 실망이었습니다.,가격과 맛에 대한 실망을 드려 죄송합니다. 더 다양한 메뉴와 향상된 품질로 보답하겠...,0,,,


In [ ]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        item = self.df.iloc[idx]
        store = item["store"]
        branch = item["branch"]
        food = item["food"]
        review = item["고객리뷰"]
        answer = item["사장댓글"]

        return {
            "store": store,
            "branch": branch,
            "food": food,
            "review": review,
            "answer": answer
        }

In [ ]:
def collate_fn(batch):
    texts = [
        f"<w>{item['store']}</s><b>{item['branch']}</s><f>{item['food']}</s><q>{item['review']}</s><a>{item['answer']}</s>"
        for item in batch
    ]
    x = tokenizer(texts, return_tensors="pt", padding=True)

    return {"x": x}

In [ ]:
dt = ChatDataset(df)
dt[0]

{'store': ' ',
 'branch': '',
 'food': '김치찌개',
 'review': '김치찌개는 별로였어요. 약간 김치국 느낌이 나서 실망했어요. 고기도 별로였고. ',
 'answer': '안녕하세요. 김치찌개와 고기에 대한 실망을 드려 죄송합니다. 지속적인 품질 개선을 위해 노력하겠습니다. 감사합니다.'}

In [ ]:
dt = ChatDataset(df)
dl = torch.utils.data.DataLoader(dt,batch_size=2,collate_fn=collate_fn)
batch = next(iter(dl))
batch

{'x': {'input_ids': tensor([[ 9724,   461,   405,   739,     1,  9724,   440,   405,     1,  9724,
            444,   405,  6963,  8420,  8326,  6841,     1,  9724,   455,   405,
           6963,  8420,  8326, 19010, 15126,  8041,  8006, 25856, 10968, 33974,
           6920, 31131, 13875, 40974,  8718,  8006, 25856,  9065,  9313, 15126,
          19570,   389,   739,     1,  9724,   439,   405,  7967,  7114,  8702,
           7801, 25856, 33974,  8326, 24655,  9065,  9399,  9167,  9137, 12615,
           9535,  7415, 11482,  7832, 37194, 18323, 14912, 32272,  9207, 10805,
          26616, 16691, 15940, 37194,     1],
         [ 9724,   461,   405,   739,     1,  9724,   440,   405,     1,  9724,
            444,   405,   739,     1,  9724,   455,   405, 13692, 12371, 27080,
           9208,  8539,  8267,  7724,  8718,  8006, 25856, 10997,  7235, 15126,
           8041,  8006, 25856,   739,     1,  9724,   439,   405,  7609,  8539,
           8267,  7724,  8702,  7816, 16179, 29205, 247

In [ ]:
model(**batch["x"]).logits.shape

torch.Size([2, 75, 51200])

# 학습루프

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in tqdm(dataloader):

        x = batch["x"].to(device)

        pred = model(**x).logits  # batch, seq, 단어별 실수값
        n_class = pred.shape[-1]  # 클래스 개수

        pred = pred[:, :-1, :]  # eos 토큰 제외
        pred = pred.reshape(-1, n_class)  # 2차원 형태로 변환

        tgt = x["input_ids"][:, 1:]
        tgt = tgt.flatten()  # 1차원 벡터 형태로 바뀜

        mask = tgt != tokenizer.pad_token_id  # 패딩토큰 제외 -> 손실을 계산할 때 제외
        tgt = tgt[mask]
        pred = pred[mask]
        loss = loss_fn(pred, tgt)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dataloader)

    return epoch_loss

In [ ]:
reset_seeds(SEED)
batch_size = 2
epochs = 1

model_name = "skt/kogpt2-base-v2"
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
loss_fn = torch.nn.CrossEntropyLoss()

train_dt = ChatDataset(df)
train_dl = torch.utils.data.DataLoader(train_dt,batch_size=batch_size,shuffle=True,collate_fn=collate_fn)

for i in range(epochs):
    train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
    print(f"{i+1}번째 epoch: {train_loss:.4f}")


  0%|          | 0/9323 [00:00<?, ?it/s]

1번째 epoch: 1.4269


# 학습모델 저장

In [ ]:
model.save_pretrained(f"{DATA_PATH}kogpt2_0920")

# 저장한 모델 로드

In [ ]:
loaded_model = AutoModelForCausalLM.from_pretrained(f"{DATA_PATH}sohyun_23762_21_loss_0.2512").to(device)

# 테스트루프

## 유사도 구하기

In [ ]:
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=825e0e8979c6ca534f98879df82d515b763aea6bfa834eeeabb9c10dccd00567
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.metrics import jaccard_score
bert_model = SentenceTransformer('jhgan/ko-sroberta-multitask')

def evaluate_similarity(input_text, generated_text, alpha=0.9): # cosine 가중치 조절
    input_embedding = bert_model.encode(input_text)
    generated_embedding = bert_model.encode(generated_text)

    # 코사인 유사도
    cosine_sim = 1 - pairwise_distances([input_embedding], [generated_embedding], metric='cosine')[0][0]

    # 자카드 유사도
    input_tokens = set(input_text.split())
    generated_tokens = set(generated_text.split())
    jaccard_sim = len(input_tokens.intersection(generated_tokens)) / len(input_tokens.union(generated_tokens))

    # 가중 평균 내보기
    weighted_sim = alpha*cosine_sim + (1 - alpha)*jaccard_sim

    return weighted_sim

In [ ]:
evaluate_similarity("맛은 있는데 양이 적어요","양이 부족하셨다니 죄송합니다. 다음에는 개선된 모습 보이도록 하겠습니다")

0.326508781042966

## 테스트루프 함수화

In [ ]:
def chatbot_with_evaluation(model, tokenizer, max_len, device, num_samples=5):
    model.eval()
    while True:
        text = input("user > ").strip()
        if text == "quit":
            break

        text = "<q>" + text + "</s><a>"
        x = tokenizer.encode(text, return_tensors="pt").to(device)
        q_len = len(text) + 1

        best_generated_text = None
        best_similarity_score = -1.0
        generated_texts = []  # 답변 후보군을 담을 리스트
        for i in range(num_samples):
            result_ids = model.generate(x,
                                        max_length=max_len,
                                        repetition_penalty=2.0,
                                        num_beams=1,
                                        num_return_sequences=1,
                                        no_repeat_ngram_size=4,
                                        use_cache=True,
                                        do_sample=True,
                                        temperature=0.8,
                                        top_k=90,
                                        top_p=0.95,
                                        early_stopping=True
                                        )
            generated_text = tokenizer.decode(result_ids[0])
            generated_text = generated_text[q_len:-4]
            generated_text = re.sub(r'[^가-힣.]', ' ', generated_text)

            similarity_score = evaluate_similarity(text, generated_text)
            generated_texts.append((similarity_score,generated_text))
            print(generated_texts[i])

            if similarity_score > best_similarity_score:
                best_similarity_score = similarity_score
                best_generated_text = generated_text

        print()
        print("Best_reply >", best_generated_text)
        print("Best_similarity_score:", best_similarity_score)
        print("--------------------------------------------------------------------------------------------")


# 추론하기

In [ ]:
import warnings
%config InlineBackend.figure_format = 'retina'
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
chatbot_with_evaluation(loaded_model,tokenizer,128,device)

user > 맛있어요
(0.49730805158615116, '고객님의 리뷰를 보고 마음이 따뜻해집니다 맛있다는 칭찬으로 가득한 리뷰 덕분에 오늘 하루도 파이팅 넘치게 시작할 수 있습니당 고객님께서 저희 매장의 음식을 드시고 싶다고 해주시니 신선한 재료와 넉넉한 마음입니다  앞으로 자주 뵈어요  다시 반가이미하게 대접할게용 왕 빈자 삼파전을 찾아주신 우리 고객님을 믿고 주문하셔서 감사합니까든 양으로만 충분하다  하트랄 게 없겠지만 다른 메뉴로 행복한 시간 보내셨기를 바랍니다.')
(0.43797930479049685, '고객님  맛있게 드셔주시고 리뷰 남겨 주신 점 감사합니다    더 노력 하겠습니다  늘 건강하시고 행복하세용 왕 빈자 삼파전을 사랑해 주시고  찾아주심에 진심으로 정성을 다해 보답하겠습니다. 이제 곧 저희도 행복한 마음으로 찾을 수 있는 가게가 되겠어요.. 울 고객님의 가정에 건강과 웃음꽃이 활짝 피기를 바라봅니다. 양도  서비스 챙겨 드리려고 항상 노력하고 있습니다. 이용해 주시는 분들께 즐거움과 행복을이 가득 담아 드릴게요  앞으로 더욱 발전하도록 최선')
(0.3472124934196472, '감사합니다. 소중한 시간 내주셔서 별점을 받으셨다니 감사합니다  항상 최선을 다하는 가게가 되도록 노력하겠습니다  정말 죄송해요. ')
(0.49064705371856693, '고객님  맛있게 드셔주시고 리뷰 남겨 주심에 감사합니다    이제 곧 저희 음식드릴 테니 자주 애용해 주세여 영양 만점 사진까지 항상 변치 않는 맛있는 음식으로 보답 드리겠습니다 앞으로도 행복한 하루 보내시길 바랄게용  또 만나세요  고객님의 정성스러운 칼럼 전할 것을 약속드리며  소중한 의견에 힘이 되는 별점과 서비스로 감동시켜 드릴 수 있도록 노력하겠습니다. 즐거운 시간 되십시오. 싱글 재미있으셨을까 봐 이런 한 말씀이 최고의 칭찬')
(0.449178010225296, '고객님  맛있게 드셔주시고 리뷰 남겨 주신 점 짱입니다    앞으로도 자주 주문해 주세합니다  고객님